In [59]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
integer_encoded=data
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example

NameError: name 'data' is not defined

In [60]:
import xlearn as xl
import pandas as pd
import numpy as np
import gc
import time
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame as DF
from keras.utils import to_categorical
from numpy import array
from sklearn.preprocessing import OneHotEncoder
import hashlib, math, os, subprocess

In [75]:
data3 = pd.read_csv('/data/data/final_data.csv',sep=',')

In [77]:
data3=data3.sort_values(['uid','timeStamp'],ascending=True)

In [78]:
data3.head()


,uid,mid,star,timeStamp
61302,1000001,1301578,1,2005.0
99298,1000001,1293590,1,2005.0
190034,1000001,1309004,1,2005.0
195159,1000001,1292063,1,2005.0
335374,1000001,1292001,1,2005.0


In [20]:
# uid 48613  mid 72541
mid = data3['mid'].unique()
uid = data3['uid'].unique()
collect_mid_encoder = {}
collect_mid_dencoder = {}
collect_uid_encoder = {}
collect_uid_dencoder = {}
for i in range(len(mid)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i
for i in range(len(uid)):
    collect_uid_encoder[i] = uid[i]
    collect_uid_dencoder[uid[i]] = i
data3['mid'] = data3['mid'].map(lambda x:collect_mid_dencoder[x]+1)
data3['uid'] = data3['uid'].map(lambda x:collect_uid_dencoder[x]+1)

In [21]:
data3.head()

,uid,mid,star,timeStamp
61302,1,1,1,2005.0
99298,1,2,1,2005.0
190034,1,3,1,2005.0
195159,1,4,1,2005.0
335374,1,5,1,2005.0


In [79]:
data3=data3.dropna()

In [23]:
data3['timeStamp']=data3['timeStamp'].apply(lambda x:int(x)-2005)

In [24]:
data3.head()

,uid,mid,star,timeStamp
61302,1,1,1,0
99298,1,2,1,0
190034,1,3,1,0
195159,1,4,1,0
335374,1,5,1,0


In [80]:
data3['last_mid']=0

In [84]:
data3.head()

,uid,mid,star,timeStamp,last_mid
61302,1000001,1301578,1,2005.0,0
99298,1000001,1293590,1,2005.0,0
190034,1000001,1309004,1,2005.0,0
195159,1000001,1292063,1,2005.0,0
335374,1000001,1292001,1,2005.0,0


In [110]:
data3.head()

,uid,mid,star,timeStamp,last_mid
61302,1000001,1301578,1,2005.0,0
99298,1000001,1293590,1,2005.0,0
190034,1000001,1309004,1,2005.0,0
195159,1000001,1292063,1,2005.0,0
335374,1000001,1292001,1,2005.0,0


In [71]:
data3.to_csv('/data/ng/FFM/last_mid.csv', header=True, index=False, sep=' ', mode='w')

In [109]:
temp_uid=0
temp_mid=0
i=0
for index,row in data3.iterrows():
    if(temp_uid==row['uid']):
        row['last_mid']=temp_mid
    else:
        temp_uid=row['uid']
    temp_mid=row['mid']
    i+=1
    if(i%10==0):
        break
        print(i/100000,end='\r')
#     break

In [93]:
data3.iloc[1]

uid          1000001.0
mid          1293590.0
star               1.0
timeStamp       2005.0
last_mid           0.0
Name: 99298, dtype: float64

In [14]:
uid_array=array(data3['uid'])
uid_encoded = to_categorical(uid_array)
mid_array=array(data3['mid'])
mid_encoded = to_categorical(mid_array)

MemoryError: 

In [18]:
mid_array=array(data3['mid'])

In [21]:
len(uid_array)

4182695

In [59]:
with open('model.out','w') as f:
    print(1)

1


In [110]:
import xlearn as xl

ffm_model = xl.create_ffm()
# ffm_model.setTrain("criteo.tr.r100.gbdt0.ffm")
ffm_model.setTrain("test.ffm")
# ffm_model.setValidate("criteo.va.r100.gbdt0.ffm")
# ffm_model.setValidate("val.ffm")

param = {'task':'binary', # ‘binary’ for classification, ‘reg’ for Regression
         'k':2,           # Size of latent factor
         'lr':0.1,        # Learning rate for GD
         'lambda':0.0002, # L2 Regularization Parameter
#          'Metric':'auc',  # Metric for monitoring validation set performance
         'epoch':25       # Maximum number of Epochs
        }

ffm_model.fit(param, "/data/ng/model.out")

In [75]:
# ffm_model.setTest("criteo.va.r100.gbdt0.ffm")
ffm_model.setTest("val.ffm")
ffm_model.setSigmoid()
ffm_model.predict("/data/ng/model.out", "output1.txt")

In [102]:
for i in range(1):
    with open('val.ffm','w') as f:
        temp=data3[data3['uid']!='0:'+str(i)+':1']['mid']
        num=len(temp)
        uid_temp=['0:'+str(i)+':1']*num
        temp_pd=pd.DataFrame(list(zip(uid_temp,temp)))
        temp_pd.to_csv(f,index=False,sep=' ')
        ffm_model.setTest("val.ffm")
        ffm_model.setSigmoid()
        ffm_model.predict("model.out", "output1.txt")
    break    
# a=[(1, 4), (2, 5), (3, 6)]
# a=['1:0:1']*3
# b=[4,5,6]
# c=list(zip(a,b))
# print(c)
# da=pd.DataFrame(c)

In [17]:
readin = open('/data/data/delete_final_data.txt', 'r')
#write data file
output = open('/data/ng/libsvm.txt', 'w')
try:
    the_line = readin.readline()
    while the_line:
        # delete the \n
        index=0
        the_line = the_line.strip('\n')
        output_line = ''
        temp=the_line.split('\t')
        print(temp)
        for i in range(len(temp)):
            #the label col
            if i == 0:
                output_line = temp[len(temp)-1]
            #the features cols
            if temp[i-1] != 'NULL' and i != 0:
                the_text = ' ' + str(i) + ':' + temp[i-1]
                output_line = output_line + the_text
        output_line = output_line + '\n'
        print(output_line)
#         index+=1
#         if(index>2):
#             break
        break
        output.write(output_line)
        the_line = readin.readline()
finally:
    readin.close()

[',uid,mid,star,timeStamp,count']
,uid,mid,star,timeStamp,count



In [45]:
data3 = pd.read_csv('/data/data/delete_final_data.txt',sep=',')

In [46]:
data3=data3.sort_values(['uid','timeStamp'],ascending=True)

In [47]:
data3.head()

,Unnamed: 0,uid,mid,star,timeStamp,count
35158,35158,1000001,1301578,1,2005.0,3482
162453,162453,1000001,1292722,1,2005.0,11883
237003,237003,1000001,1291832,1,2005.0,8533
497857,497857,1000001,1291561,1,2005.0,12225
705804,705804,1000001,1292720,1,2005.0,14474


In [48]:
mid = data3['mid'].unique()
uid = data3['uid'].unique()
collect_mid_encoder = {}
collect_mid_dencoder = {}
collect_uid_encoder = {}
collect_uid_dencoder = {}
for i in range(len(mid)):
    collect_mid_encoder[i] = mid[i]
    collect_mid_dencoder[mid[i]] = i
for i in range(len(uid)):
    collect_uid_encoder[i] = uid[i]
    collect_uid_dencoder[uid[i]] = i
data3['mid'] = data3['mid'].map(lambda x:collect_mid_dencoder[x])
data3['uid'] = data3['uid'].map(lambda x:collect_uid_dencoder[x])

In [49]:
data3.drop(labels=['Unnamed: 0'], axis=1,inplace = True)
data3.head()

,uid,mid,star,timeStamp,count
35158,0,0,1,2005.0,3482
162453,0,1,1,2005.0,11883
237003,0,2,1,2005.0,8533
497857,0,3,1,2005.0,12225
705804,0,4,1,2005.0,14474


In [50]:
data3=data3.dropna()
data3['timeStamp']=data3['timeStamp'].apply(lambda x:int(x)-2005)

In [51]:
data3['uid']=data3['uid'].apply(lambda x:'0:'+str(x)+':1')
data3['mid']=data3['mid'].apply(lambda x:'1:'+str(x)+':1')
data3['timeStamp']=data3['timeStamp'].apply(lambda x:'2:'+str(x)+':1')

In [52]:
data3.drop(labels=['count'], axis=1,inplace = True)
data3.head()

,uid,mid,star,timeStamp
35158,0:0:1,1:0:1,1,2:0:1
162453,0:0:1,1:1:1,1,2:0:1
237003,0:0:1,1:2:1,1,2:0:1
497857,0:0:1,1:3:1,1,2:0:1
705804,0:0:1,1:4:1,1,2:0:1


In [5]:
def my_one_hot(num,MAX,MIN=0):
    tag=[0]*(MAX-MIN+1)
    tag[num]=1
    return array(tag)

In [29]:
uid_array=array(data3['uid'])
mid_array=array(data3['mid'])
max_uid=max(uid_array)
min_uid=min(uid_array)
max_mid=max(mid_array)
min_mid=min(mid_array)

In [73]:
da2 = pd.read_csv('/data/ng/FFM/last_mid.csv',sep=' ')

In [74]:
da2.head()

,uid,mid,star,timeStamp,last_mid
0,1,1,1,0,0
1,1,2,1,0,1
2,1,3,1,0,2
3,1,4,1,0,3
4,1,5,1,0,4


In [53]:
star = data3['star']
data3.drop(labels=['star'], axis=1,inplace = True)
data3.insert(0, 'label', star)

In [54]:
data3.head()

,label,uid,mid,timeStamp
35158,1,0:0:1,1:0:1,2:0:1
162453,1,0:0:1,1:1:1,2:0:1
237003,1,0:0:1,1:2:1,2:0:1
497857,1,0:0:1,1:3:1,2:0:1
705804,1,0:0:1,1:4:1,2:0:1


In [55]:
data3.to_csv('test.txt', header=False, index=False, sep=' ', mode='w')

In [111]:
a=[(1, 4), (2, 5), (3, 6)]
a=['1:0:1']*3
b=[4,5,6]
c=list(zip(a,b))
print(c)
da=pd.DataFrame(c)

[('1:0:1', 4), ('1:0:1', 5), ('1:0:1', 6)]


In [15]:
t=my_one_hot(1,50000,0)

In [69]:
str='1'
nr_bins=1e+6
ttt=int(hashlib.md5(str.encode('utf8')).hexdigest(), 16) % (int(nr_bins) - 1) + 1
print(ttt)

428585


In [112]:
da.head()

,0,1
0,1:0:1,4
1,1:0:1,5
2,1:0:1,6


In [118]:
da[2]=da[0]+da[1]

TypeError: 'str' object is not callable